# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [4]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../../modules')
import hull_white as hw

import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [5]:
df_curva = pd.read_excel('../../data/20201012_built_sofr_zero.xlsx')

In [6]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [7]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')
print()

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [8]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [9]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [10]:
gamma = 1
sigma = .005

In [11]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [29]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [30]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después).


Pregunta 2:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a -.1772 .


Pregunta 3:

Ceteris paribus una opción sobre una activo con reversión a la media vale menos
que una opción sobre un activo sin reversión a la media. Comente.


Pregunta 4:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 1282 y 2454 días.


Pregunta 5:

Considere un swap a 1Y con cupones semestrales de 1.3% (lineal). Si la tasa
cupón cero a 0.5Y es 1.2% (exp), calcule la tasa cero a 1Y.


Pregunta 6:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.3%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 7:

Expliqu

## Respuestas

### Pregunta 1

In [ ]:
'''
Calcule la TNA con 
ICP0 = 10,000.00  
ICP365 = 10,049.00 (365 días después).
'''

In [35]:
ICP0 = 10000.00 
ICP365 = 10049.00 
d=365

# formula TNA
#𝑇𝑁𝐴=𝑅𝑜𝑢𝑛𝑑((𝐼𝐶𝑃_T/𝐼𝐶𝑃_𝑡−1)⋅360/d,4)

TNA=round(((ICP365/ICP0)-1)*360/d,4)
print(TNA)

0.0048


### Pregunta 2

In [ ]:
'''
Calcule un paso de simulación de Montecarlo para el modelo de Vasicek 
usando 
gamma = 1.0, 
sigma = 0.5%, 
dt = 1/264, 
r0 obtenido de la curva 
cupón cero entregada y número aleatorio N(0, 1) igual a -.1772 .
'''

In [52]:

gamma=1.0
sigma = 0.5/100
dt = 1/264
r_=0.000781   #tasa de largo plazo
r0=0.000811   #tasa inicial (t = 0  -> r0 = 0.000811) de la simulación
num_dias=33   # num_dias: número de días en la trayectoria, incluyendo el instante t = 0
#pasos_dia=1  #pasos_dia: número de pasos de simulación en 1 día 

In [12]:

def vasicek_path(r0: float,
                 gamma: float, 
                 r_: float,
                 sigma: float,
                 num_dias: int = 33,
                 dias_agno: int = 264,
                 pasos_dia: int = 1): # -> list[tuple[float, float]]
    """
    Retorna un camino de simulación del modelo de Vasicek.
    
    params:
    
    - r0: tasa inicial (t = 0) de la simulación
    - Parámetros del modelo:
      - gamma: velocidad de reversión
      - r_: tasa de largo plazo
      - sigma: volatilidad
    - num_dias: número de días en la trayectoria, incluyendo el instante t = 0
    - dias_agno: número de días hábiles por año
    - pasos_dia: número de pasos de simulación en 1 día
    
    return:
    
    - Un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.
    """
    dt = 1 / (dias_agno * pasos_dia)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    gamma_dt = gamma * dt
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        #r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * np.random.normal() # Discretización de Euler
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * -.1772
        result.append((i * dt, r))
    return result

In [57]:
test = np.random.default_rng(-.1772)
test.normal()  

TypeError: SeedSequence expects int or sequence of ints for entropy not -0.1772

### Pregunta 3

In [ ]:
'''
Ceteris paribus una opción sobre una activo con reversión a 
la media vale menos que una opción sobre un activo sin 
reversión a la media.
Comente.
'''

# ceteris paribus -> todo lo demas es constante

In [ ]:
Sabemos que

- la volatilidad realizada (en los modelos con reversion a la media) no 
depende solamente del elemento de difusion, sino que depende de la 
fuerza que tenga el driff


Para una misma volatilidad local valen más las opciones sin reversión a 
la media ya que la reversion disminuye la volatilidad  -> esto dado el 
gamma que se encuentra en la funcion de bonos cupon cero


### Pregunta 4

In [ ]:
'''
Pregunta 4:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la 
tasa forward entre 1282 y 2454 días.
'''


In [58]:
from scipy.interpolate import interp1d

In [59]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')  

In [60]:
t1 = 1282 
t2 = 2454 
r1 = curva(t1) 
r2 = curva(t2)
df1 = math.exp(-r1 * t1 / 365)
df2 = math.exp(-r2 * t2 / 365)
fwd12 = (df1 / df2 -1)*360 / (t2 - t1)
print(fwd12)

0.0059331987373331795


### Pregunta 5

In [ ]:
'''
Considere un swap a 1Y con cupones semestrales de 1.3% (lineal). Si la 
tasa cupón cero a 0.5Y es 1.2% (exp), calcule la tasa cero a 1Y.
'''

### Pregunta 6

In [ ]:
'''
Usando la curva cero cupón entregada y parámetros 
gamma = 1.0 y 
sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y 
con un strike de 99.3%.

Entregue el resultado 8 decimales y suponiendo un nocional de 1.
'''

In [ ]:
gamma = 1
sigma = .005

In [66]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


In [77]:
def zrate(t: float) -> float:
    return zcurva(t)

def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta

def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta


In [78]:

def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [79]:
r0 = zrate(0)
def theta(t: float) -> float:
    # theta es nivel de reversión a la media variable en el tiempo
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux


NameError: name 'zcurva' is not defined

In [72]:
thetas = []
dt = 1 / 365.0
for i in range(0, 8000):
    t = i * dt
    thetas.append((i * dt, theta(t)))
df_theta = pd.DataFrame(thetas, columns=['t', 'theta'])


NameError: name 'theta' is not defined

In [ ]:
#Función A(t) del Modelo HW
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    Calcula el valor de la función A(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - zrate: curva cero cupón
    - fwd: Tasa forward
    - gamma: intensidad de reversión del modelo HW
    - sigma: volatilidad
    - t: cuando quiero calcular
    - T: plazo del bono

    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [ ]:
#Función B(t) del Modelo HW
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t: cuando quiero calcular
    - T: plazo del bono
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [ ]:
#Bono Cero Cupón 
def zero_hw(r: float,
            gamma: float,
            sigma: float,
            zrate: float,
            fwd: float,
            t: float,
            T: float) -> float:
    """
    Calculo Bono Cupón Cero
    params:
    - r: tasa simulada
    - gamma: intensidad de reversión del modelo HW
    - sigma: volatilidad
    - zrate: curva cero cupón
    - fwd: Tasa forward
    - t: cuando quiero calcular
    - T: plazo del bono
    """
 
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [ ]:
#PRECIOS BONO CERO CUPÓN EN t DE PLAZO T A PARTIR DE 1000 TASAS CORTAS SIMULADAS
precios = [zero_hw(r, gamma, sigma, zrate, fwd, t, T) for r in last_rates]
P = pd.DataFrame()
P["PRECIOS"]=precios
#P

#no esta en valor presente por que para esto debo calcular el valor esperado 
#del valor presente y necesito el recorrido de cada simulación

In [ ]:
def Vanilla_Payoff(z: float, strike: float, tipo) -> float:
    """
    Cálculo payoff de una call o put 
    
   - z: es el precio del bono cupón cero al vencimiento
   - strike: es el strike de la opción
   - Tipo: es el tipo de opción 
   """
    eps=1
        
    return max(max(eps*(z - strike), .000), .000)

In [ ]:
payoffs_C = {strike:[Vanilla_Payoff(z, strike, 'Call') for z in precios] for strike in strikes_C}

In [ ]:
#FACTORES DE DESCUENTO
df = np.array(math.exp(-dt * np.sum(s[0])))

for sim in s[1:]:
    df=np.append(df, math.exp(-dt * np.sum(sim)))
                                              

In [ ]:
# CALL
Vp_C = {strike:[np.average(np.multiply(df,payoffs_C[strike]))] for strike in strikes_C}
print(Vp_C)

### Pregunta 7

In [ ]:
'''
Explique la diferencia entre valorizar un payoff g(r(T), T) por 
simulación de Montecarlo utilizando el modelo de Hull-White en la 
medida libre de riesgo y el modelo de Hull-White en la medida T-forward.
'''

In [ ]:
Sabemos que:

Por montecarlo
- Al hacer una simulacion genero una trayectoria posible de tasas en el
futuro donde en cada uno de esos puntos de la simulacion tengo una curva
cero cupon
-El problema que tengo es que al querer calcular el payoff no puedo 
sacar el factor de descuento del valor esperado ya que la tasa es 
estocastica (por lo que no podemos asumir una tasa constante y traerlo a 
valor presente)
-Cuando tenemos heterogeneidad de instrumentos necesitamos traer a valor
presente por la trayectoria


Pro la medida T-foward
-Se utiliza el drift
- La curva se obtiene interpolando linealmente en los factores de 
descuento
-se extrae la tasa de la ultima simulacion.. no es necesario la trayectoria